In [ ]:
# 1️⃣ Setup: Clone repo và mount Drive
!git clone https://github.com/dungdinhhaha/AIDetect.git /content/ComparisonDetector 2>/dev/null || (cd /content/ComparisonDetector && git pull)
%cd /content/ComparisonDetector

from google.colab import drive
drive.mount("/content/drive")

print("✓ Setup completed")

In [ ]:
# 2️⃣ Test data pipeline
import os
import tensorflow as tf
from configs.config_v2 import ConfigV2
from data.loader_tf2 import build_dataset

cfg = ConfigV2()
tfrecord_paths = tf.io.gfile.glob(os.path.join(cfg.DATA_DIR, "*.tfrecord"))
print(f"Found {len(tfrecord_paths)} TFRecord files: {tfrecord_paths}")

ds = build_dataset(tfrecord_paths, image_size=cfg.IMAGE_SIZE, batch_size=cfg.BATCH_SIZE)

for images, targets in ds.take(1):
    print(f"\n✓ Data pipeline working!")
    print(f"  Images shape: {images.shape}")
    print(f"  Boxes shape: {targets['boxes'].shape}")
    print(f"  Labels shape: {targets['labels'].shape}")

In [ ]:
# 3️⃣ 🚀 Start Training
!python train_keras.py

In [ ]:
# 4️⃣ 📊 TensorBoard (optional)
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/comparison_detector_models_v2/logs

In [ ]:
# 5️⃣ Evaluate model
model_path = os.path.join(cfg.MODEL_DIR, 'model.keras')
if os.path.exists(model_path):
    loaded_model = tf.keras.models.load_model(model_path)
    print(f"✓ Model loaded from {model_path}")
    
    # Test inference
    test_img = tf.random.uniform((1, 640, 640, 3))
    pred = loaded_model(test_img, training=False)
    print(f"  Prediction shape: {pred.shape}")
    print(f"  Predicted class: {tf.argmax(pred[0]).numpy()}")
else:
    print(f"⚠ Model not found at {model_path}")